In [130]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

spark = SparkSession.builder.appName('abc').getOrCreate()

In [131]:
categorias = ['action','adaptation','adventure','apocalypse','artistic',\
              'assassination','based on a true story','biblical','blood','brutal','biographical','bollywood','boring',\
              'cars','cerebral','classic','censorship','comedy','computers','confusing',\
              'cooking','comic','cartoon','court','crime','cult','dark','death','disaster','documentary','drama',
              'depressing','drugs','environment','erotic','fantasy','fighting','football','freedom',\
              'friendship','genius','god','gothic','high school','historical','hollywood','horror',\
              'humor','homosexuality','holiday','independent film','kids','love','magic','marriage',\
              'military','murder','musical','nature','nostalgia','nudity','olympics','original','oscar',\
              'pirates','police','pornography','prison','prostitution','psychology','realistic',\
              'revolution','robot','romance','scary','science','sex','snakes','soccer','space','sports',\
              'spy','story','stunning','superhero','surreal','suspense','technology','teen','thriller',\
              'time','torture','tragedy','travel','treasure','true story','utopia','3d','war','wizards',\
              'zombie','dreamworks','disney','gore','imdb top 250','literary adaptation','mafia','pixar','sci fi','violence'
             ]

In [132]:
genome_tags_path = "../../ml-latest/genome-tags.csv"
ratings_path = "../../ml-latest/ratings.csv"
genome_scores_path = "../../ml-latest/genome-scores.csv"

In [133]:
genome_tags = spark.read.csv(genome_tags_path,header='true')
ratings = spark.read.csv(ratings_path,header='true')
genome_scores = spark.read.csv(genome_scores_path,header='true')

In [134]:
genome_tags_filtrado = genome_tags.where(f.col("tag").isin(categorias))

In [135]:
etiquetas_df = genome_scores.join(genome_tags_filtrado,['tagId'],'leftsemi')

In [136]:
etiquetas_relevancia = etiquetas_df.join(genome_tags_filtrado,['tagId'],'left')

In [137]:
completa = etiquetas_relevancia.groupby("movieId").pivot("tag").agg(f.avg('relevance'))

In [138]:
metricas_ratings = ratings.groupby("movieId").agg(f.avg(f.col("rating")).alias("promedio_rating"),
                                                  f.count(f.col("userId")).alias("conteo_usuarios"))

In [139]:
completa_metricas = completa.join(metricas_ratings,['movieId'], 'left').drop("movieId")

In [140]:
from pyspark.ml.feature import VectorAssembler

columnas_input = list(completa_metricas.columns)
columnas_input.remove('promedio_rating')

vectorAssembler = VectorAssembler(inputCols = columnas_input, outputCol = 'features')
completa_metricas_vector = vectorAssembler.transform(completa_metricas)

splits = completa_metricas_vector.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

In [141]:
from pyspark.ml.regression import LinearRegression
import mlflow

mlflow.set_tracking_uri("http://192.168.1.68:5000")
maxIter = 100
elasticNetParam = 0.1
regParam=0.1
                        
lr = LinearRegression(featuresCol='features',labelCol='promedio_rating',maxIter=maxIter,elasticNetParam=elasticNetParam,regParam=regParam)

with mlflow.start_run():
    mlflow.log_param("maxIter", maxIter)
    mlflow.log_param("elasticNetParam", elasticNetParam)
    mlflow.log_param("regParam", regParam)


    lr_model = lr.fit(train_df)
    #print("Coefficients: " + str(lr_model.coefficients))
    #print("Intercept: " + str(lr_model.intercept))
    trainingSummary = lr_model.summary
    print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
    print("r2: %f" % trainingSummary.r2)

    mlflow.log_metric("rmse", trainingSummary.rootMeanSquaredError)
    mlflow.log_metric("r2", trainingSummary.r2)

ConnectionError: HTTPConnectionPool(host='192.168.1.68', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0534aed410>: Failed to establish a new connection: [Errno 110] Connection timed out'))